<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [40]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
import spacy
import re
from gensim import matutils, models, corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim
import scipy.sparse
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# read amazon reviews csv
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head(10)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."
5,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Bought a lot of batteries for Christmas and th...,... batteries for Christmas and the AmazonBasi...,ByPainter Marlow,"https://www.barcodable.com/upc/841710106442,ht..."
6,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,ive not 

In [3]:
df['reviews.text'].head()

0    I order 3 of them and one of the item is bad q...
1    Bulk is always the less expensive way to go fo...
2    Well they are not Duracell but for the price i...
3    Seem to work as well as name brand batteries a...
4    These batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [4]:
# creating a function to pull out nouns, verbs and adjectives from a string of text

def nouns_adj_vb(text):
    '''Given a string of text, tokenize the text and pull out only the nouns, adjectives and verbs.'''
    is_noun_adj_vb = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'VB'
    tokenized = word_tokenize(text)
    nouns_adj_vb = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj_vb(pos)] 
    return ' '.join(nouns_adj_vb)

In [5]:
df['reviews']= df['reviews.text'].apply(nouns_adj_vb)

In [6]:
df['reviews'].head()

0    order item is bad quality Is missing backup sp...
1                    Bulk is expensive way go products
2                        are Duracell price i am happy
3          Seem work name brand batteries better price
4                 batteries are lasting price is great
Name: reviews, dtype: object

In [7]:
df.shape

(28332, 25)

In [8]:
df.primaryCategories.unique()

array(['Health & Beauty', 'Electronics', 'Office Supplies',
       'Animals & Pet Supplies', 'Home & Garden', 'Electronics,Furniture',
       'Toys & Games,Electronics', 'Electronics,Media',
       'Office Supplies,Electronics'], dtype=object)

In [9]:
amazon_reviews = df[['primaryCategories','reviews']]
amazon_reviews.head()

,primaryCategories,reviews
0,Health & Beauty,order item is bad quality Is missing backup sp...
1,Health & Beauty,Bulk is expensive way go products
2,Health & Beauty,are Duracell price i am happy
3,Health & Beauty,Seem work name brand batteries better price
4,Health & Beauty,batteries are lasting price is great


In [10]:
amazon_reviews.set_index('primaryCategories')

,reviews
primaryCategories,
Health & Beauty,order item is bad quality Is missing backup sp...
Health & Beauty,Bulk is expensive way go products
Health & Beauty,are Duracell price i am happy
Health & Beauty,Seem work name brand batteries better price
Health & Beauty,batteries are lasting price is great
...,...
Electronics,got old twins yr old has is Perfect way get read
Electronics,bought niece Christmas gift.she is years old love
Electronics,nice light internet browsing keeping top email...


In [11]:
stop_words = text.ENGLISH_STOP_WORDS.union(['amazon'])
cvw = CountVectorizer(stop_words= stop_words, max_df =.9)
data_cvw = cvw.fit_transform(amazon_reviews.reviews)

In [12]:
data_cvw.check_format

<bound method _cs_matrix.check_format of <28332x9427 sparse matrix of type '<class 'numpy.int64'>'
	with 277823 stored elements in Compressed Sparse Row format>>

In [13]:
data_cvw.shape

(28332, 9427)

In [14]:
data_dtm = pd.DataFrame(data_cvw.todense(), columns = cvw.get_feature_names())
data_dtm.index = amazon_reviews.index
data_dtm

,00,01,02,09500043,10,100,100v,10th,11,111,...,äôs,äôsadvertisement,äôt,äôve,äù,äùtablet,äúdual,äúparent,äúyou,ôºå
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Create the gensim corpus
corpus = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtm.transpose()))

# Create the vocabulary dictionary
id2word = dict((v, k) for k, v in cvw.vocabulary_.items())

# # A Dictionary Representation of all the words in our corpus
# id2word = corpora.Dictionary(amazon_reviews['reviews.text'])
# # a bag of words(bow) representation of our corpus
# # Note: we haven't actually read any text into memory here
# corpus = [id2word.doc2bow(text) for text in amazon_reviews['reviews.text']]

In [17]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 5,
                   passes=10,
                   workers=4
                  )

In [18]:
lda.print_topics()

[(0,
  '0.045*"kindle" + 0.013*"light" + 0.013*"battery" + 0.012*"read" + 0.012*"screen" + 0.011*"love" + 0.011*"new" + 0.011*"reading" + 0.010*"reader" + 0.010*"easy"'),
 (1,
  '0.039*"tablet" + 0.032*"great" + 0.016*"screen" + 0.013*"price" + 0.013*"use" + 0.012*"good" + 0.012*"apps" + 0.010*"love" + 0.009*"works" + 0.009*"books"'),
 (2,
  '0.076*"batteries" + 0.045*"great" + 0.041*"good" + 0.039*"price" + 0.021*"work" + 0.015*"buy" + 0.014*"battery" + 0.013*"brand" + 0.012*"product" + 0.011*"value"'),
 (3,
  '0.056*"tablet" + 0.033*"loves" + 0.030*"great" + 0.028*"use" + 0.027*"bought" + 0.027*"easy" + 0.025*"old" + 0.017*"good" + 0.017*"gift" + 0.016*"year"'),
 (4,
  '0.055*"kids" + 0.036*"love" + 0.029*"great" + 0.019*"tablet" + 0.019*"year" + 0.019*"games" + 0.016*"old" + 0.014*"bought" + 0.014*"apps" + 0.013*"use"')]

In [22]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [23]:
for id, t in enumerate(topics):
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
kindle light battery read screen

------ Topic 1 ------
tablet great screen price use

------ Topic 2 ------
batteries great good price work

------ Topic 3 ------
tablet loves great use bought

------ Topic 4 ------
kids love great tablet year



In [25]:
pyLDAvis.enable_notebook()

In [55]:
# id2word_final = corpora.Dictionary(amazon_reviews.reviews)
# pyLDAvis.gensim.prepare(lda, corpus, id2word)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling